# Verification of New Results for Bots Case
#### Unified Model with Integrated Bot 

James Yu, 12 January 2023

In [1]:
import matplotlib.pyplot as plt
import numpy as np
from numpy.linalg import inv
np.set_printoptions(suppress=True) # disable scientific notation for readability

In [2]:
def infinite_solution(A_n, A_c, delta, c, x_0, z, b = np.identity(3)):
    n = len(x_0)
    eps = np.finfo(np.float64).eps
    I = np.identity(n)
    O = np.zeros((n, n))
    
    Q = np.block([
        [I, O], 
        [O, I]
    ])
    A = np.block([
        [A_n, A_c], 
        [O, I]
    ])
    B = np.block([
        [b], # identity matrix by default 
        [O]
    ])
    K_t = Q
    K_sequence = [K_t]
    
    while True: # generate solution matrices; note I divide delta into the inverse term for simplification
        K_t_new = Q + delta * A.T @ (K_t - K_t @ B @ inv(B.T @ K_t @ B + c * I / delta) @ B.T @ K_t) @ A
        K_sequence.insert(0, K_t_new)
        if np.allclose(K_t, K_t_new, rtol = eps, atol = eps): break
        K_t = K_t_new

    chi_var = np.block([
        [x_0], 
        [z]
    ])
    chi_ts = [chi_var]
    r_ts = []
    K_ss = K_sequence[0]
    L_ss = -inv(B.T @ K_ss @ B + c * I / delta) @ B.T @ K_ss @ A
    
    payoff = 0
    stage_payoffs = []
    discounted_stage_payoffs = []
    cumulative_payoffs = []
    
    i = 0
    while True:
        r_t = L_ss @ chi_var
        r_ts.append(r_t)
        p = -(chi_var.T @ Q @ chi_var + c * r_t.T @ r_t + z.T @ z).item()
        payoff += delta**i * p
        stage_payoffs.append(p)
        discounted_stage_payoffs.append(delta**i * p)
        cumulative_payoffs.append(payoff)
        chi_var_new = A @ chi_var + B @ r_t
        chi_ts.append(chi_var_new)
        if np.allclose(chi_var, chi_var_new, rtol = eps, atol = eps): break
        chi_var = chi_var_new
        i += 1
        
    return chi_ts, r_ts, stage_payoffs, discounted_stage_payoffs, cumulative_payoffs, K_ss, L_ss

# 0. Setup

In [3]:
A_1_n = 0.99 * np.array([
    [0.3, 0.4, 0.3],
    [0.6, 0.2, 0.2],
    [0.5, 0.1, 0.4]
])
A_1_c = np.diag([0.01, 0.01, 0.01])

delta = 0.9
c = 1.0
x_0 = np.array([[10.0, 5.0, -20.0]], ndmin = 2).T
z = np.array([[5.0, 5.0, 5.0]], ndmin = 2).T

In [4]:
chi_ts, r_ts, stage_payoffs, discounted_stage_payoffs, cumulative_payoffs, K_ss, L_ss = infinite_solution(A_1_n, A_1_c, delta, c, x_0, z)

In [5]:
print(K_ss)

[[ 1.38616486  0.17105178  0.23321578  0.00371302  0.0044536   0.00413393]
 [ 0.17105178  1.11998571  0.11887399  0.00315167  0.00201313  0.00154197]
 [ 0.23321578  0.11887399  1.16516367  0.00293632  0.00209553  0.00321817]
 [ 0.00371302  0.00315167  0.00293632 10.00070065  0.00022898  0.00020687]
 [ 0.0044536   0.00201313  0.00209553  0.00022898 10.00055207  0.00005221]
 [ 0.00413393  0.00154197  0.00321817  0.00020687  0.00005221 10.0006539 ]]


In [6]:
print(L_ss)

[[-0.20416138 -0.22807784 -0.18698905 -0.00678598 -0.00198875 -0.00195085]
 [-0.316895   -0.11369141 -0.11730197 -0.00157713 -0.00571962 -0.00073902]
 [-0.2773602  -0.07228275 -0.21818719 -0.0015369  -0.00094037 -0.00629405]]


In [7]:
chi_ts[-1] # terminal opinions

array([[-0.00103162],
       [ 0.01107937],
       [ 0.00755896],
       [ 5.        ],
       [ 5.        ],
       [ 5.        ]])

In [8]:
r_ts[-1] # terminal messages

array([[-0.05735767],
       [-0.04199823],
       [-0.04602059]])

# 1. Expression for $K_1$ component of the block matrix $K^*$

Recall that this expression was:

$$
K_1 = I + c\delta(A^n)'K_1(\delta K_1 + cI)^{-1}A^n
$$

In [9]:
print(K_ss)

[[ 1.38616486  0.17105178  0.23321578  0.00371302  0.0044536   0.00413393]
 [ 0.17105178  1.11998571  0.11887399  0.00315167  0.00201313  0.00154197]
 [ 0.23321578  0.11887399  1.16516367  0.00293632  0.00209553  0.00321817]
 [ 0.00371302  0.00315167  0.00293632 10.00070065  0.00022898  0.00020687]
 [ 0.0044536   0.00201313  0.00209553  0.00022898 10.00055207  0.00005221]
 [ 0.00413393  0.00154197  0.00321817  0.00020687  0.00005221 10.0006539 ]]


In [10]:
print(K_ss[0:3,0:3]) # K_1

[[1.38616486 0.17105178 0.23321578]
 [0.17105178 1.11998571 0.11887399]
 [0.23321578 0.11887399 1.16516367]]


In [11]:
K_1 = K_ss[0:3,0:3]
LHS = np.identity(3) + c*delta*A_1_n.T @ K_1 @ inv(delta*K_1 + c*np.identity(3)) @ A_1_n
np.allclose(LHS, K_1) # is approximately equal?

True

In [12]:
LHS

array([[1.38616486, 0.17105178, 0.23321578],
       [0.17105178, 1.11998571, 0.11887399],
       [0.23321578, 0.11887399, 1.16516367]])

They appear to be the same. To decompose:

In [13]:
c*delta*A_1_n.T @ K_1

array([[0.56586377, 0.69742486, 0.64496902],
       [0.54529011, 0.27113598, 0.20811753],
       [0.4841214 , 0.28767028, 0.49878625]])

In [14]:
inv(delta*K_1 + c*np.identity(3)) 

array([[ 0.45129744, -0.03222578, -0.04455476],
       [-0.03222578,  0.50170187, -0.0228987 ],
       [-0.04455476, -0.0228987 ,  0.49388767]])

In [15]:
c*delta*A_1_n.T @ K_1 @ inv(delta*K_1 + c*np.identity(3)) @ A_1_n

array([[0.38616486, 0.17105178, 0.23321578],
       [0.17105178, 0.11998571, 0.11887399],
       [0.23321578, 0.11887399, 0.16516367]])

# 2. Expression for $K_2$ component of the block matrix $K^*$

Recall this was:

$$
K_2 = (A^n)'c\delta K_1 (\delta K_1 + cI)^{-1}A^z + (A^n)'(\delta K_1 + cI)^{-1}K_2
$$

In [16]:
print(K_ss)

[[ 1.38616486  0.17105178  0.23321578  0.00371302  0.0044536   0.00413393]
 [ 0.17105178  1.11998571  0.11887399  0.00315167  0.00201313  0.00154197]
 [ 0.23321578  0.11887399  1.16516367  0.00293632  0.00209553  0.00321817]
 [ 0.00371302  0.00315167  0.00293632 10.00070065  0.00022898  0.00020687]
 [ 0.0044536   0.00201313  0.00209553  0.00022898 10.00055207  0.00005221]
 [ 0.00413393  0.00154197  0.00321817  0.00020687  0.00005221 10.0006539 ]]


In [17]:
print(K_ss[0:3, 3:6]) # K_2

[[0.00371302 0.0044536  0.00413393]
 [0.00315167 0.00201313 0.00154197]
 [0.00293632 0.00209553 0.00321817]]


In [18]:
K_2 = K_ss[0:3, 3:6]
LHS_2 = A_1_n.T @ (c * delta * K_1) @ inv(delta * K_1 + c*np.identity(3)) @ A_1_c + A_1_n.T @ inv(delta * K_1 + c*np.identity(3)) @ K_2
np.allclose(LHS_2, K_2) # is approximately equal?

False

In [19]:
LHS_2

array([[0.00389873, 0.00459633, 0.00428508],
       [0.00324844, 0.00211048, 0.00163299],
       [0.00305481, 0.00219803, 0.00333332]])

# 3. Expression for $x_{\infty}$ terminal opinions

$$
x_\infty = (\delta K_1 + cI - cA^n)^{-1}(cA^z - \delta K_2)z
$$

In [20]:
LHS_3 = inv(delta * K_1 + c*np.identity(3) - c * A_1_n) @ (c * A_1_c - delta * K_2) @ z
LHS_3

array([[-0.00103162],
       [ 0.01107937],
       [ 0.00755896]])

In [21]:
chi_ts[-1][0:3]

array([[-0.00103162],
       [ 0.01107937],
       [ 0.00755896]])

In [22]:
np.allclose(chi_ts[-1][0:3], LHS_3)

True